# WordPiece tokenization

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

Also log into Hugging Face.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

WordPiece is the <font color='blue'>tokenization algorithm</font> Google developed to <font color='blue'>pretrain BERT</font>. It has since been reused in quite a few <font color='blue'>Transformer models</font> based on BERT, such as <font color='blue'>DistilBERT</font>, <font color='blue'>MobileBERT</font>, <font color='blue'>Funnel Transformers</font>, and <font color='blue'>MPNET</font>. It's very similar to BPE in terms of the training, but the actual <font color='blue'>tokenization</font> is <font color='blue'>done differently</font>.

<Tip>

💡 This section covers WordPiece in depth, going as far as showing a full implementation. You can skip to the end if you just want a general overview of the tokenization algorithm.

</Tip>

## Training algorithm

<Tip warning={true}>

⚠️ Google never open-sourced its implementation of the training algorithm of WordPiece, so what follows is our best guess based on the published literature. It may not be 100% accurate.

</Tip>

Like BPE, WordPiece starts from a <font color='blue'>small vocabulary</font> including the <font color='blue'>special tokens</font> used by the <font color='blue'>model</font> and the <font color='blue'>initial alphabet</font>. Since it identifies <font color='blue'>subwords</font> by <font color='blue'>adding</font> a <font color='blue'>prefix</font> (like `##` for BERT), each <font color='blue'>word</font> is initially <font color='blue'>split</font> by <font color='blue'>adding that prefix</font> to <font color='blue'>all</font> the <font color='blue'>characters</font> inside the <font color='blue'>word</font>. So, for instance, `"word"` gets split like this:

```
w ##o ##r ##d
```

Thus, the <font color='blue'>initial alphabet</font> contains <font color='blue'>all</font> the <font color='blue'>characters</font> present at the <font color='blue'>beginning</font> of a word and the characters present <font color='blue'>inside a word</font> preceded by the WordPiece prefix.

Then, again like BPE, WordPiece learns <font color='blue'>merge rules</font>. The <font color='blue'>main difference</font> is the <font color='blue'>way</font> the <font color='blue'>pair to be merged</font> is <font color='blue'>selected</font>. Instead of selecting the most frequent pair, WordPiece <font color='blue'>computes a score</font> for <font color='blue'>each pair</font>, using the following formula:

$$\mathrm{score} = \frac{\mathrm{freq\_of\_pair}}{ \mathrm{freq\_of\_first\_element} \times \mathrm{freq\_of\_second\_element}}$$

By <font color='blue'>dividing</font> the <font color='blue'>frequency of the pair</font> by the <font color='blue'>product of the frequencies of each of its parts</font>, the algorithm <font color='blue'>prioritizes</font> the merging of pairs where the <font color='blue'>individual parts are less frequent</font> in the vocabulary. For instance, it <font color='blue'>won't</font> necessarily <font color='blue'>merge `("un", "##able")`</font> even if that pair <font color='blue'>occurs very frequently</font> in the <font color='blue'>vocabulary</font>, because the two pairs `"un"` and `"##able"` will likely each appear in a lot of other words and have a high frequency. In contrast, a pair like <font color='blue'>`("hu", "##gging")`</font> will probably be <font color='blue'>merged faster</font> (assuming the word "hugging" appears often in the vocabulary) since <font color='blue'>`"hu"`</font> and <font color='blue'>`"##gging"`</font> are likely to be <font color='blue'>less frequent</font> individually.

Let's look at the same vocabulary we used in the BPE training example:

```
("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
```

The splits here will be:

```
("h" "##u" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("h" "##u" "##g" "##s", 5)
```

so the initial vocabulary will be `["b", "h", "p", "##g", "##n", "##s", "##u"]` (if we forget about special tokens for now). The most frequent pair is <font color='blue'>`("##u", "##g")`</font> (present <font color='blue'>20</font> times), but the individual frequency of `"##u"` is very high, so its score is not the highest (it's <font color='blue'>1 / 36</font>). All pairs with a <font color='blue'>`"##u"`</font> actually have that <font color='blue'>same score (1 / 36)</font>, so the best score goes to the pair <font color='blue'>`("##g", "##s")`</font> -- the only one without a `"##u"` -- at <font color='blue'>1 / 20</font>, and the <font color='blue'>first merge learned</font> is <font color='blue'>`("##g", "##s") -> ("##gs")</font>`.

Note that when we <font color='blue'>merge</font>, we <font color='blue'>remove</font> the <font color='blue'>`##` between</font> the <font color='blue'>two tokens</font>, so we <font color='blue'>add `"##gs"`</font> to the <font color='blue'>vocabulary</font> and <font color='blue'>apply the merge</font> in the words of the corpus:

```
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs"]
Corpus: ("h" "##u" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("h" "##u" "##gs", 5)
```

At this point, <font color='blue'>`"##u"`</font> is in <font color='blue'>all</font> the <font color='blue'>possible pairs</font>, so they <font color='blue'>all</font> end up with the <font color='blue'>same score</font>. Let's say that in this case, the first pair is merged, so <font color='blue'>`("h", "##u") -> "hu"`</font>. This takes us to:

```
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs", "hu"]
Corpus: ("hu" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("hu" "##gs", 5)
```

Then the next best score is shared by <font color='blue'>`("hu", "##g")` and `("hu", "##gs")`</font> (with <font color='blue'>1/15</font>, compared to <font color='blue'>1/21</font> for all the other pairs), so the <font color='blue'>first pair</font> with the <font color='blue'>biggest score</font> is <font color='blue'>merged</font>:

```
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs", "hu", "hug"]
Corpus: ("hug", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("hu" "##gs", 5)
```

and we continue like this until we reach the desired vocabulary size.

<Tip>

✏️ **Now your turn!** What will the next merge rule be?

</Tip>

The next merge rule will be <font color='blue'>("hu", "##gs") -> "hugs"</font>.
Among the remaining pairs, ("hu", "##gs") has the highest score of 1/15, compared to other pairs like ("p", "##u"), ("##u", "##g"), and ("##u", "##n") which all have lower scores of 1/21 due to the high individual frequency of "##u". The corpus becomes:

```
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs", "hu", "hug", "hugs"]
Corpus: ("hug", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("hugs", 5)
```

## Tokenization algorithm
Tokenization differs in WordPiece and BPE in that <font color='blue'>WordPiece</font> only <font color='blue'>saves</font> the <font color='blue'>final vocabulary</font>, <font color='blue'>not</font> the <font color='blue'>merge rules</font> learned. Starting from the word to tokenize, <font color='blue'>WordPiece</font> finds the <font color='blue'>longest subword</font> that is <font color='blue'>in</font> the <font color='blue'>vocabulary</font>, then splits on it. For instance, if we use the vocabulary learned in the example above, for the word <font color='blue'>`"hugs"`</font> the <font color='blue'>longest subword</font> starting from the beginning that is inside the vocabulary is <font color='blue'>`"hug"`</font>, so we <font color='blue'>split there</font> and get <font color='blue'>`["hug", "##s"]`</font>. We then <font color='blue'>continue</font> with <font color='blue'> `"##s"`</font>, which is in the vocabulary, so the <font color='blue'>tokenization</font> of <font color='blue'>`"hugs"`</font> is <font color='blue'>`["hug", "##s"]`</font>.

With <font color='blue'>BPE</font>, we would have applied the merges learned in order and <font color='blue'>tokenized</font> this <font color='blue'>as `["hu", "##gs"]`</font>, so the encoding is different.

As <font color='blue'>another example</font>, let's see how the word <font color='blue'>`"bugs"`</font> would be <font color='blue'>tokenized</font>. <font color='blue'>`"b"`</font> is the <font color='blue'>longest subword</font> starting at the beginning of the word that is in the vocabulary, so we split there and get <font color='blue'>`["b", "##ugs"]`</font>. Then <font color='blue'>`"##u"`</font> is the <font color='blue'>longest subword</font> starting at the <font color='blue'>beginning</font> of <font color='blue'>`"##ugs"`</font> that is <font color='blue'>in</font> the <font color='blue'>vocabulary</font>, so we split there and get <font color='blue'>`["b", "##u, "##gs"]`</font>. Finally, `"##gs"` is in the vocabulary, so this last list is the tokenization of `"bugs"`.

When the tokenization gets to a stage where it's <font color='blue'>not possible</font> to find a <font color='blue'>subword</font> in the <font color='blue'>vocabulary</font>, the <font color='blue'>whole word</font> is <font color='blue'>tokenized</font> as <font color='blue'>unknown</font> -- so, for instance, <font color='blue'>`"mug"`</font> would be <font color='blue'>tokenized</font> as <font color='blue'>`["[UNK]"]`</font>, as would <font color='blue'>`"bum"`</font> (even if we can begin with `"b"` and `"##u"`, `"##m"` is not the vocabulary, and the resulting tokenization will just be `["[UNK]"]`, not `["b", "##u", "[UNK]"]`). This is another difference from <font color='blue'>BPE</font>, which would <font color='blue'>only classify</font> the <font color='blue'>individual characters</font> not in the vocabulary as <font color='blue'>unknown</font>.

<Tip>

✏️ **Now your turn!** How will the word `"pugs"` be tokenized?

</Tip>

For the word <font color='blue'>"pugs"</font>, we start by finding the <font color='blue'>longest subword</font> from the <font color='blue'>beginning</font> that is in the vocabulary. <font color='blue'>"p"</font> is the longest subword starting at the beginning of the word that is in the vocabulary, so we <font color='blue'>split there</font> and get <font color='blue'>["p", "##ugs"]</font>. Then <font color='blue'>"##u"</font> is the <font color='blue'>longest subword</font> starting at the <font color='blue'>beginning</font> of <font color='blue'>"##ugs"</font> that is in the vocabulary, so we <font color='blue'>split there</font> and get <font color='blue'>["p", "##u", "##gs"]</font>. Then, since <font color='blue'>"##gs"</font> is <font color='blue'>in</font> the <font color='blue'>vocabulary</font>, the tokenization of "pugs" is ["p", "##u", "##gs"].

## Implementing WordPiece

Now let's take a look at an <font color='blue'>implementation</font> of the <font color='blue'>WordPiece algorithm</font>. Like with BPE, this is just pedagogical, and you won't able to use this on a big corpus.

We will use the same corpus as in the BPE example:

In [ ]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

<font color='blue'></font>, we need to <font color='blue'>pre-tokenize</font> the <font color='blue'>corpus</font> into <font color='blue'>words</font>. Since we are replicating a WordPiece tokenizer (like BERT), we will use the `bert-base-cased` tokenizer for the pre-tokenization:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Then we <font color='blue'>compute</font> the <font color='blue'>frequencies</font> of <font color='blue'>each word</font> in the corpus as we do the pre-tokenization:

In [ ]:
from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

word_freqs

defaultdict(int,
            {'This': 3,
             'is': 2,
             'the': 1,
             'Hugging': 1,
             'Face': 1,
             'Course': 1,
             '.': 4,
             'chapter': 1,
             'about': 1,
             'tokenization': 1,
             'section': 1,
             'shows': 1,
             'several': 1,
             'tokenizer': 1,
             'algorithms': 1,
             'Hopefully': 1,
             ',': 1,
             'you': 1,
             'will': 1,
             'be': 1,
             'able': 1,
             'to': 1,
             'understand': 1,
             'how': 1,
             'they': 1,
             'are': 1,
             'trained': 1,
             'and': 1,
             'generate': 1,
             'tokens': 1})

As we saw before, the <font color='blue'>alphabet</font> is the <font color='blue'>unique set</font> composed of all the <font color='blue'>first letters</font> of <font color='blue'>words</font>, and <font color='blue'>all</font> the <font color='blue'>other letters</font> that appear in words <font color='blue'>prefixed</font> by <font color='blue'>`##`</font>:

In [ ]:
alphabet = []
for word in word_freqs.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])
    for letter in word[1:]:
        if f"##{letter}" not in alphabet:
            alphabet.append(f"##{letter}")

alphabet.sort()
alphabet

print('[' + ',\n '.join(f"'{element}'" for element in alphabet) + ']')

['##a',
 '##b',
 '##c',
 '##d',
 '##e',
 '##f',
 '##g',
 '##h',
 '##i',
 '##k',
 '##l',
 '##m',
 '##n',
 '##o',
 '##p',
 '##r',
 '##s',
 '##t',
 '##u',
 '##v',
 '##w',
 '##y',
 '##z',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'g',
 'h',
 'i',
 's',
 't',
 'u',
 'w',
 'y']


We also add the <font color='blue'>special tokens</font> used by the model at the <font color='blue'>beginning</font> of that <font color='blue'>vocabulary</font>. In the case of BERT, it's the list `["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]`:


In [ ]:
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()

Next we need to <font color='blue'>split</font> each <font color='blue'>word</font>, with all the <font color='blue'>letters</font> that are <font color='blue'>not</font> the <font color='blue'>first prefixed</font> by <font color='blue'>`##`</font>:

In [ ]:
splits = {
    word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
    for word in word_freqs.keys()
}

Now that we are ready for training, let's write a <font color='blue'>function</font> that <font color='blue'>computes</font> the <font color='blue'>score</font> of <font color='blue'>each pair</font>. We'll need to use this at each step of the training:

In [ ]:
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq

    scores = {
        pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return scores

Let's have a look at a <font color='blue'>part</font> of <font color='blue'>this dictionary</font> after the initial splits:

In [ ]:
pair_scores = compute_pair_scores(splits)
for i, key in enumerate(pair_scores.keys()):
    print(f"{key}: {pair_scores[key]}")
    if i >= 5:
        break

('T', '##h'): 0.125
('##h', '##i'): 0.03409090909090909
('##i', '##s'): 0.02727272727272727
('i', '##s'): 0.1
('t', '##h'): 0.03571428571428571
('##h', '##e'): 0.011904761904761904


Now, finding the <font color='blue'>pair</font> with the <font color='blue'>best score</font> only takes a quick <font color='blue'>loop</font>:

In [ ]:
best_pair = ""
max_score = None
for pair, score in pair_scores.items():
    if max_score is None or max_score < score:
        best_pair = pair
        max_score = score

print(best_pair, max_score)

('a', '##b') 0.2


So the <font color='blue'>first merge</font> to learn is <font color='blue'>`('a', '##b') -> 'ab'`</font>, and we add `'ab'` to the vocabulary:


In [ ]:
vocab.append("ab")

To continue, we need to apply that <font color='blue'>merge</font> in our <font color='blue'>`splits` dictionary</font>. Let's write another function for this:

In [ ]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                split = split[:i] + [merge] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

And we can have a look at the <font color='blue'>result</font> of the <font color='blue'>first merge</font>:

In [ ]:
splits = merge_pair("a", "##b", splits)
splits["about"]

['ab', '##o', '##u', '##t']

Now we have everything we need to <font color='blue'>loop</font> until we have <font color='blue'>learned all</font> the <font color='blue'>merges</font> we want. Let's aim for a vocab size of <font color='blue'>70</font>:

In [ ]:
vocab_size = 70
while len(vocab) < vocab_size:
    scores = compute_pair_scores(splits)
    best_pair, max_score = "", None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score
    splits = merge_pair(*best_pair, splits)
    new_token = (
        best_pair[0] + best_pair[1][2:]
        if best_pair[1].startswith("##")
        else best_pair[0] + best_pair[1]
    )
    vocab.append(new_token)


We can then look at the generated vocabulary:

In [ ]:
print('[' + ',\n '.join(f"'{element}'" for element in vocab) + ']')

['[PAD]',
 '[UNK]',
 '[CLS]',
 '[SEP]',
 '[MASK]',
 '##a',
 '##b',
 '##c',
 '##d',
 '##e',
 '##f',
 '##g',
 '##h',
 '##i',
 '##k',
 '##l',
 '##m',
 '##n',
 '##o',
 '##p',
 '##r',
 '##s',
 '##t',
 '##u',
 '##v',
 '##w',
 '##y',
 '##z',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'g',
 'h',
 'i',
 's',
 't',
 'u',
 'w',
 'y',
 'ab',
 '##fu',
 'Fa',
 'Fac',
 '##ct',
 '##ful',
 '##full',
 '##fully',
 'Th',
 'ch',
 '##hm',
 'cha',
 'chap',
 'chapt',
 '##thm',
 'Hu',
 'Hug',
 'Hugg',
 'sh',
 'th',
 'is',
 '##thms',
 '##za',
 '##zat',
 '##ut']


As we can see, compared to BPE, this tokenizer learns <font color='blue'>parts of words</font> as <font color='blue'>tokens</font> a bit <font color='blue'>faster</font>.

<Tip>

💡 Using <font color='blue'>`train_new_from_iterator()`</font> on the <font color='blue'>same corpus won't result</font> in the exact <font color='blue'>same vocabulary</font>. This is because the 🤗 Tokenizers library does not implement WordPiece for the training (since we are not completely sure of its internals), but uses BPE instead.

</Tip>

To <font color='blue'>tokenize</font> a <font color='blue'>new text</font>, we <font color='blue'>pre-tokenize</font> it, <font color='blue'>split</font> it, then <font color='blue'>apply</font> the <font color='blue'>tokenization algorithm</font> on <font color='blue'>each word</font>. That is, we look for the <font color='blue'>biggest subword starting</font> at the <font color='blue'>beginning</font> of the <font color='blue'>first word</font> and <font color='blue'>split it</font>, then we <font color='blue'>repeat the process</font> on the second part, and so on for the <font color='blue'>rest</font> of <font color='blue'>that word and</font> the <font color='blue'>following words</font> in the text:


In [ ]:
def encode_word(word):
    tokens = []
    while len(word) > 0:
        i = len(word)
        while i > 0 and word[:i] not in vocab:
            i -= 1
        if i == 0:
            return ["[UNK]"]
        tokens.append(word[:i])
        word = word[i:]
        if len(word) > 0:
            word = f"##{word}"
    return tokens

Let's test it on <font color='blue'>one word</font> that's in the <font color='blue'>vocabulary</font>, and <font color='blue'>another</font> that <font color='blue'>isn't</font>:

In [ ]:
print(encode_word("Hugging"))
print(encode_word("HOgging"))

['Hugg', '##i', '##n', '##g']
['[UNK]']


Now, let's write a <font color='blue'>function</font> that <font color='blue'>tokenizes</font> a <font color='blue'>text</font>:

In [ ]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    encoded_words = [encode_word(word) for word in pre_tokenized_text]
    return sum(encoded_words, [])

We can try it on <font color='blue'>any text</font>:

In [ ]:
tokenize("This is the Hugging Face course!")

['Th',
 '##i',
 '##s',
 'is',
 'th',
 '##e',
 'Hugg',
 '##i',
 '##n',
 '##g',
 'Fac',
 '##e',
 'c',
 '##o',
 '##u',
 '##r',
 '##s',
 '##e',
 '[UNK]']

That's it for the WordPiece algorithm! Now let's take a look at Unigram.